<h1><center>Toronto and Montreal K-means Clusters</center></h1>

This notebook contains my code for the project of the IBM-Coursera Applied Data Science course. 

The notebook contains a lot of data structuring that was web-scrapped. Hence, it was divided to provide orientation to the readers. 

Sections:

1. Section 1: Data Collection and Structuring
 1. Montreal Data Scrapping and Structuring
 2. Toronto Data Collection 
2. Section 2: Montreal Zip Codes Clustering
3. Section 3: Toronto Zip Codes Clustering
4. Section 4: Montreal and Toronto Clustering as One Dataset

A descriptive statistics notebook will be uploaded separetly. 

In [10]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Section 1: Data Collection and Structuring

 ### Montreal Data Scrapping and Structuring

In [11]:
url = 'https://www.geonames.org/postal-codes/CA/QC/quebec.html'

df = pd.read_html(url)

len(df)

3

In [12]:
df[2].head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Mont-Joli,G5H,Canada,Quebec,Bas-Saint-Laurent,Mont-Joli
1,NaN,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192
2,2.0,Duvernay-Est,H7A,Canada,Quebec,NaN,NaN
3,NaN,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592
4,3.0,Saint-Vincent-de-Paul,H7C,Canada,Quebec,NaN,NaN


In [13]:
df = df[2]

df2 = df

dfnew = df2.fillna(0)

dfnew.head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Mont-Joli,G5H,Canada,Quebec,Bas-Saint-Laurent,Mont-Joli
1,0.0,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192
2,2.0,Duvernay-Est,H7A,Canada,Quebec,0,0
3,0.0,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592
4,3.0,Saint-Vincent-de-Paul,H7C,Canada,Quebec,0,0


In [14]:
dfnew.columns = ['Number', 'Place', 'Code', 'Country', 'Admin1', 'Admin2', 'Admin3']

dfnew = dfnew.drop(dfnew[dfnew['Number'] == 0].index)

dfnew.reset_index(drop=True)

,Number,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Mont-Joli,G5H,Canada,Quebec,Bas-Saint-Laurent,Mont-Joli
1,2.0,Duvernay-Est,H7A,Canada,Quebec,0,0
2,3.0,Saint-Vincent-de-Paul,H7C,Canada,Quebec,0,0
3,4.0,Duvernay,H7E,Canada,Quebec,0,0
4,5.0,Pont-Viau,H7G,Canada,Quebec,0,0
5,6.0,Auteuil West,H7H,Canada,Quebec,Auteuil,0
6,7.0,Auteuil Northeast,H7J,Canada,Quebec,Auteuil,0
7,8.0,Auteuil South,H7K,Canada,Quebec,Auteuil,0
8,9.0,Sainte-Rose,H7L,Canada,Quebec,0,0
9,10.0,Vimont,H7M,Canada,Quebec,0,0


In [15]:
dfnew = dfnew.drop(columns = 'Number')

dfnew.head()

,Place,Code,Country,Admin1,Admin2,Admin3
0,Mont-Joli,G5H,Canada,Quebec,Bas-Saint-Laurent,Mont-Joli
2,Duvernay-Est,H7A,Canada,Quebec,0,0
4,Saint-Vincent-de-Paul,H7C,Canada,Quebec,0,0
6,Duvernay,H7E,Canada,Quebec,0,0
8,Pont-Viau,H7G,Canada,Quebec,0,0


In [16]:
dfnew = dfnew.reset_index(drop=True)

In [17]:
import re

dfmontreal2 = dfnew.loc[dfnew['Code'].str.contains('^H[a-z]*', flags=re.I, regex=True)]

dfmontreal2.head()

,Place,Code,Country,Admin1,Admin2,Admin3
1,Duvernay-Est,H7A,Canada,Quebec,0,0
2,Saint-Vincent-de-Paul,H7C,Canada,Quebec,0,0
3,Duvernay,H7E,Canada,Quebec,0,0
4,Pont-Viau,H7G,Canada,Quebec,0,0
5,Auteuil West,H7H,Canada,Quebec,Auteuil,0


In [18]:
dfmontreal2 = dfmontreal2.drop(columns = ['Country', 'Admin3'])

dfmontreal2.head()

,Place,Code,Admin1,Admin2
1,Duvernay-Est,H7A,Quebec,0
2,Saint-Vincent-de-Paul,H7C,Quebec,0
3,Duvernay,H7E,Quebec,0
4,Pont-Viau,H7G,Quebec,0
5,Auteuil West,H7H,Quebec,Auteuil


In [19]:
dfmontreal2 = dfmontreal2.reset_index(drop=True)

dfmontreal2

,Place,Code,Admin1,Admin2
0,Duvernay-Est,H7A,Quebec,0
1,Saint-Vincent-de-Paul,H7C,Quebec,0
2,Duvernay,H7E,Quebec,0
3,Pont-Viau,H7G,Quebec,0
4,Auteuil West,H7H,Quebec,Auteuil
5,Auteuil Northeast,H7J,Quebec,Auteuil
6,Auteuil South,H7K,Quebec,Auteuil
7,Sainte-Rose,H7L,Quebec,0
8,Vimont,H7M,Quebec,0
9,Laval-des-Rapides,H7N,Quebec,Laval


In [20]:
dfmontreal2 = dfmontreal2.drop(16)

dfmontreal2

,Place,Code,Admin1,Admin2
0,Duvernay-Est,H7A,Quebec,0
1,Saint-Vincent-de-Paul,H7C,Quebec,0
2,Duvernay,H7E,Quebec,0
3,Pont-Viau,H7G,Quebec,0
4,Auteuil West,H7H,Quebec,Auteuil
5,Auteuil Northeast,H7J,Quebec,Auteuil
6,Auteuil South,H7K,Quebec,Auteuil
7,Sainte-Rose,H7L,Quebec,0
8,Vimont,H7M,Quebec,0
9,Laval-des-Rapides,H7N,Quebec,Laval


In [21]:
dfmontreal2 = dfmontreal2.reset_index(drop=True)

In [22]:
dfmontreal2.head()

,Place,Code,Admin1,Admin2
0,Duvernay-Est,H7A,Quebec,0
1,Saint-Vincent-de-Paul,H7C,Quebec,0
2,Duvernay,H7E,Quebec,0
3,Pont-Viau,H7G,Quebec,0
4,Auteuil West,H7H,Quebec,Auteuil


In [23]:
df2.columns = ['Number', 'Place', 'Code', 'Country', 'Admin1', 'Admin2', 'Admin3']

dfcode = df2.fillna(0)

dfcode = dfcode.iloc[0::2, :]

dfcode.head()

,Number,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Mont-Joli,G5H,Canada,Quebec,Bas-Saint-Laurent,Mont-Joli
2,2.0,Duvernay-Est,H7A,Canada,Quebec,0,0
4,3.0,Saint-Vincent-de-Paul,H7C,Canada,Quebec,0,0
6,4.0,Duvernay,H7E,Canada,Quebec,0,0
8,5.0,Pont-Viau,H7G,Canada,Quebec,0,0


In [24]:
dfgeo = df2.fillna(0)

dfgeo = dfgeo.iloc[1::2, :]

dfgeo.head()

,Number,Place,Code,Country,Admin1,Admin2,Admin3
1,0.0,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192,48.584/-68.192
3,0.0,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592,45.674/-73.592
5,0.0,45.617/-73.649,45.617/-73.649,45.617/-73.649,45.617/-73.649,45.617/-73.649,45.617/-73.649
7,0.0,45.623/-73.695,45.623/-73.695,45.623/-73.695,45.623/-73.695,45.623/-73.695,45.623/-73.695
9,0.0,45.577/-73.687,45.577/-73.687,45.577/-73.687,45.577/-73.687,45.577/-73.687,45.577/-73.687


In [25]:
dfgeo = dfgeo.drop(columns = ['Number', 'Code', 'Country', 'Admin1', 'Admin2', 'Admin3'])

dfgeo.columns = ['Geo']
dfgeo.head()

,Geo
1,48.584/-68.192
3,45.674/-73.592
5,45.617/-73.649
7,45.623/-73.695
9,45.577/-73.687


In [26]:
dfgeo = dfgeo.reset_index(drop=True)

In [27]:
dfcode = dfcode.drop(columns = ['Number', 'Place', 'Country', 'Admin1', 'Admin2', 'Admin3'])

dfcode = dfcode.reset_index(drop=True)

In [28]:
dfgeo.head()

,Geo
0,48.584/-68.192
1,45.674/-73.592
2,45.617/-73.649
3,45.623/-73.695
4,45.577/-73.687


In [29]:
dfcode.head()

,Code
0,G5H
1,H7A
2,H7C
3,H7E
4,H7G


In [30]:
dfcode['C'] = dfcode.reset_index().index

dfcode.head()

,Code,C
0,G5H,0
1,H7A,1
2,H7C,2
3,H7E,3
4,H7G,4


In [31]:
dfgeo['C'] = dfgeo.reset_index().index

dfgeo.head()

,Geo,C
0,48.584/-68.192,0
1,45.674/-73.592,1
2,45.617/-73.649,2
3,45.623/-73.695,3
4,45.577/-73.687,4


In [32]:
dfcode = dfcode.merge(dfgeo)

dfcode.head()

,Code,C,Geo
0,G5H,0,48.584/-68.192
1,H7A,1,45.674/-73.592
2,H7C,2,45.617/-73.649
3,H7E,3,45.623/-73.695
4,H7G,4,45.577/-73.687


In [33]:
dfmontreal2 = dfcode.merge(dfmontreal2)

dfmontreal2.head()

,Code,C,Geo,Place,Admin1,Admin2
0,H7A,1,45.674/-73.592,Duvernay-Est,Quebec,0
1,H7C,2,45.617/-73.649,Saint-Vincent-de-Paul,Quebec,0
2,H7E,3,45.623/-73.695,Duvernay,Quebec,0
3,H7G,4,45.577/-73.687,Pont-Viau,Quebec,0
4,H7H,5,45.641/-73.754,Auteuil West,Quebec,Auteuil


In [34]:
dfmontreal2 = dfmontreal2.drop(columns = ['C'])

dfmontreal2.head()

,Code,Geo,Place,Admin1,Admin2
0,H7A,45.674/-73.592,Duvernay-Est,Quebec,0
1,H7C,45.617/-73.649,Saint-Vincent-de-Paul,Quebec,0
2,H7E,45.623/-73.695,Duvernay,Quebec,0
3,H7G,45.577/-73.687,Pont-Viau,Quebec,0
4,H7H,45.641/-73.754,Auteuil West,Quebec,Auteuil


In [35]:
dfexpand = dfmontreal2['Geo'].str.split(pat = "/", expand=True)

dfexpand['C'] = dfexpand.reset_index().index

dfexpand.head()

,0,1,C
0,45.674,-73.592,0
1,45.617,-73.649,1
2,45.623,-73.695,2
3,45.577,-73.687,3
4,45.641,-73.754,4


In [36]:
dfmontreal2['C'] = dfmontreal2.reset_index().index

dfmontreal2.head()

,Code,Geo,Place,Admin1,Admin2,C
0,H7A,45.674/-73.592,Duvernay-Est,Quebec,0,0
1,H7C,45.617/-73.649,Saint-Vincent-de-Paul,Quebec,0,1
2,H7E,45.623/-73.695,Duvernay,Quebec,0,2
3,H7G,45.577/-73.687,Pont-Viau,Quebec,0,3
4,H7H,45.641/-73.754,Auteuil West,Quebec,Auteuil,4


In [37]:
dfmontreal2 = dfmontreal2.merge(dfexpand)

In [38]:
dfmontreal2 = dfmontreal2.rename(columns={"0": "Latitude", "1": "Longitude"})

dfmontreal2 = dfmontreal2.drop(columns = ['C', 'Geo'])

dfmontreal2.head()

,Code,Place,Admin1,Admin2,0,1
0,H7A,Duvernay-Est,Quebec,0,45.674,-73.592
1,H7C,Saint-Vincent-de-Paul,Quebec,0,45.617,-73.649
2,H7E,Duvernay,Quebec,0,45.623,-73.695
3,H7G,Pont-Viau,Quebec,0,45.577,-73.687
4,H7H,Auteuil West,Quebec,Auteuil,45.641,-73.754


In [45]:
dfmontreal2.columns = ['Code', 'Place', 'Province', 'Admin', 'Latitude', 'Longitude']

In [46]:
dfmontreal2.head()

,Code,Place,Province,Admin,Latitude,Longitude
0,H7A,Duvernay-Est,Quebec,0,45.674,-73.592
1,H7C,Saint-Vincent-de-Paul,Quebec,0,45.617,-73.649
2,H7E,Duvernay,Quebec,0,45.623,-73.695
3,H7G,Pont-Viau,Quebec,0,45.577,-73.687
4,H7H,Auteuil West,Quebec,Auteuil,45.641,-73.754


In [41]:
dfmontreal2.dtypes

Code         object
Province     object
Admin2       object
Place        object
Latitude     object
Longitude    object
dtype: object

In [42]:
dfmontreal2['Latitude'] = pd.to_numeric(dfmontreal2['Latitude'],errors='coerce')

dfmontreal2['Longitude'] = pd.to_numeric(dfmontreal2['Longitude'],errors='coerce')

dfmontreal2.dtypes

Code          object
Province      object
Admin2        object
Place         object
Latitude     float64
Longitude    float64
dtype: object

In [43]:
addressM = 'Montreal, Quebec'

geolocator = Nominatim(user_agent="montreal_explorer")
locationM = geolocator.geocode(addressM)
lat = locationM.latitude
long = locationM.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(lat, long))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


In [47]:
map_montreal = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, code, place in zip(dfmontreal2['Latitude'], dfmontreal2['Longitude'], dfmontreal2['Code'], dfmontreal2['Place']):
    label = '{}, {}'.format(place, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal

### Toronto Data Collection

In [51]:
torgeodf = pd.read_csv('Toronto_Geodata')

torgeodf.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [52]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [53]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, code, neighborhood in zip(torgeodf['Latitude'], torgeodf['Longitude'], torgeodf['Postal Code'], torgeodf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Section 2: Montreal Clustering

The position of places with Zip Code H8P and H4Y appear to be in the incorrect position. Thus, I will drop them so that venue around those out-of-position areas are not collected from the Foursquare API.

In [ ]:
dfmontreal2 = dfmontreal2.drop(83)

dfmontreal2 = dfmontreal2.drop(88)

In [56]:
dfmontreal2 = dfmontreal2.reset_index(drop=True)

In [209]:
CLIENT_ID = 'EKZROQDPENYHDQFSFCKZAHOS5RIX3MN2B42IYCM2MKGSQBE2' 
CLIENT_SECRET = 'MTZYRYGQOBRNPEJ3LNVYV4G5SUE41WFZU04BRJSVTKGISUWV' 
VERSION = '20180605' 

In [61]:
radius = 500
LIMIT = 100

venues = []

for lat, long, code, admin, place in zip(dfmontreal2['Latitude'], dfmontreal2['Longitude'], dfmontreal2['Code'], dfmontreal2['Admin'], dfmontreal2['Place']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            code, 
            admin,
            place,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [62]:
venuesdf = pd.DataFrame(venues)

venuesdf.columns = ['PostalCode', 'Admin', 'Place', 'PlaceLatitude', 'PlaceLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venuesdf.shape)
venuesdf.head()

(1450, 9)


,PostalCode,Admin,Place,PlaceLatitude,PlaceLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,H7A,0,Duvernay-Est,45.674,-73.592,Pharmacie Jean Coutu,45.670998,-73.589204,Pharmacy
1,H7A,0,Duvernay-Est,45.674,-73.592,Couche-Tard,45.673132,-73.594680,Convenience Store
2,H7A,0,Duvernay-Est,45.674,-73.592,Torii sushi,45.673518,-73.595423,Sushi Restaurant
3,H7A,0,Duvernay-Est,45.674,-73.592,Couche-Tard,45.672975,-73.594859,Convenience Store
4,H7C,0,Saint-Vincent-de-Paul,45.617,-73.649,Galerie D'art St Vincent de Paul,45.615210,-73.646228,Art Gallery


In [64]:
venuesdf.groupby(["Place"]).count()

,PostalCode,Admin,PlaceLatitude,PlaceLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Place,,,,,,,,
Ahuntsic Central,18,18,18,18,18,18,18,18
Ahuntsic East,4,4,4,4,4,4,4,4
Ahuntsic Southeast,18,18,18,18,18,18,18,18
Ahuntsic Southwest,14,14,14,14,14,14,14,14
Anjou East,6,6,6,6,6,6,6,6
Anjou West,9,9,9,9,9,9,9,9
Auteuil South,7,7,7,7,7,7,7,7
Auteuil West,4,4,4,4,4,4,4,4
Cartierville Central,3,3,3,3,3,3,3,3


In [65]:
len(venuesdf['VenueCategory'].unique())

245

In [66]:
# one hot encoding
montreal_onehot = pd.get_dummies(venuesdf[['VenueCategory']], prefix="", prefix_sep="")

# add Place column back to dataframe
montreal_onehot['Place'] = venuesdf['Place']

In [68]:
montreal_onehot = montreal_onehot.set_index('Place').reset_index()

In [72]:
montreal_onehot.head()

,Place,ATM,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Sausage Shop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Soccer Field,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,Duvernay-Est,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [73]:
montreal_onehot.shape

(1450, 246)

In [155]:
montreal_grouped = montreal_onehot.groupby(["Place"]).mean().reset_index()

print(montreal_grouped.shape)
montreal_grouped.head()

(91, 246)


,Place,ATM,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Sausage Shop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Soccer Field,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,Ahuntsic Central,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.055556,0.000000,0.055556,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.055556,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0

In [79]:
num_top_venues = 5

for place in montreal_grouped['Place']:
    print("----"+place+"----")
    temp = montreal_grouped[montreal_grouped['Place'] == place].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ahuntsic Central----
                venue  freq
0                Café  0.11
1  Italian Restaurant  0.06
2         Pizza Place  0.06
3    Sushi Restaurant  0.06
4         Coffee Shop  0.06


----Ahuntsic East----
                venue  freq
0                Park  0.75
1  Athletics & Sports  0.25
2                 ATM  0.00
3  Persian Restaurant  0.00
4         Music Store  0.00


----Ahuntsic Southeast----
                    venue  freq
0      Italian Restaurant  0.11
1  Furniture / Home Store  0.11
2        Department Store  0.06
3       Electronics Store  0.06
4             Fish Market  0.06


----Ahuntsic Southwest----
            venue  freq
0        Pharmacy  0.14
1    Liquor Store  0.07
2      Restaurant  0.07
3      Beer Store  0.07
4  Farmers Market  0.07


----Anjou East----
                 venue  freq
0    Convenience Store  0.33
1  American Restaurant  0.17
2          Pizza Place  0.17
3            BBQ Joint  0.17
4         Hockey Arena  0.17


----Anjou West----
     

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Place']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


places_venues_sorted = pd.DataFrame(columns=columns)
places_venues_sorted['Place'] = montreal_grouped['Place']

for ind in np.arange(montreal_grouped.shape[0]):
    places_venues_sorted.iloc[ind, 1:] = return_most_common_venues(montreal_grouped.iloc[ind, :], num_top_venues)

places_venues_sorted.head()

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahuntsic Central,Café,Bar,Breakfast Spot,Sushi Restaurant,Organic Grocery,Restaurant,Bakery,Grocery Store,Coffee Shop,Dessert Shop
1,Ahuntsic East,Park,Athletics & Sports,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
2,Ahuntsic Southeast,Furniture / Home Store,Italian Restaurant,Clothing Store,Park,Café,Breakfast Spot,Liquor Store,Electronics Store,Sandwich Place,Bank
3,Ahuntsic Southwest,Pharmacy,Café,Wine Bar,New American Restaurant,Beer Store,Bakery,Ice Cream Shop,Farmers Market,Breakfast Spot,Grocery Store
4,Anjou East,Convenience Store,Pizza Place,BBQ Joint,American Restaurant,Hockey Arena,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant


In [86]:
# set number of clusters
kclusters = 5

montreal_grouped_clustering = montreal_grouped.drop('Place', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(montreal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 1, 0, 0, 1, 0, 0], dtype=int32)

In [ ]:
# add clustering labels
places_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [106]:
montreal_merged = dfmontreal2


montreal_merged = montreal_merged.join(places_venues_sorted.set_index('Place'), on='Place')

In [109]:
montreal_merged = montreal_merged.dropna()

montreal_merged.head(10)

,Code,Place,Province,Admin,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,H7A,Duvernay-Est,Quebec,0,45.674,-73.592,1.0,Convenience Store,Sushi Restaurant,Pharmacy,Drugstore,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,H7C,Saint-Vincent-de-Paul,Quebec,0,45.617,-73.649,0.0,Breakfast Spot,Hockey Arena,Art Gallery,Yoga Studio,Farmers Market,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant
2,H7E,Duvernay,Quebec,0,45.623,-73.695,0.0,Home Service,Ski Area,Yoga Studio,Drugstore,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,H7G,Pont-Viau,Quebec,0,45.577,-73.687,1.0,Breakfast Spot,Indian Restaurant,Convenience Store,Supermarket,Park,Curling Ice,Cycle Studio,Creperie,Flower Shop,Fish Market
4,H7H,Auteuil West,Quebec,Auteuil,45.641,-73.754,1.0,ATM,Pharmacy,Grocery Store,Convenience Store,Curling Ice,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant
6,H7K,Auteuil South,Quebec,Auteuil,45.621,-73.740,0.0,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Supermarket,Dessert Shop,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant
7,H7L,Sainte-Rose,Quebec,0,45.617,-73.782,0.0,Fast Food Restaurant,Coffee Shop,Portuguese Restaurant,Breakfast Spot,Beach,Asian Restaurant,Empanada Restaurant,English Restaurant,Falafel Restaurant,Farmers Market
8,H7M,Vimont,Quebec,0,45.598,-73.716,0.0,Fast Food Restaurant,Skating Rink,Pizza Place,Yoga Studio,Drugstore,Fish Market,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Falafel Restaurant
9,H7N,Laval-des-Rapides,Quebec,Laval,45.550,-73.699,2.0,Park,Scenic Lookout,Business Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
10,H7P,Fabreville,Quebec,0,45.578,-73.800,0.0,Construction & Landscaping,Home Service,Gas Station,Paper / Office Supplies Store,Tailor Shop,Ice Cream Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [111]:
montreal_merged = montreal_merged.reset_index(drop=True)

In [112]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(montreal_merged['Latitude'], montreal_merged['Longitude'], montreal_merged['Place'], montreal_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [196]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 0, montreal_merged.columns[[1] + list(range(6, montreal_merged.shape[1]))]]

,Place,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Saint-Vincent-de-Paul,0.0,Breakfast Spot,Hockey Arena,Art Gallery,Yoga Studio,Farmers Market,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant
2,Duvernay,0.0,Home Service,Ski Area,Yoga Studio,Drugstore,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
5,Auteuil South,0.0,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Supermarket,Dessert Shop,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant
6,Sainte-Rose,0.0,Fast Food Restaurant,Coffee Shop,Portuguese Restaurant,Breakfast Spot,Beach,Asian Restaurant,Empanada Restaurant,English Restaurant,Falafel Restaurant,Farmers Market
7,Vimont,0.0,Fast Food Restaurant,Skating Rink,Pizza Place,Yoga Studio,Drugstore,Fish Market,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Falafel Restaurant
9,Fabreville,0.0,Construction & Landscaping,Home Service,Gas Station,Paper / Office Supplies Store,Tailor Shop,Ice Cream Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
10,Chomedey Northeast,0.0,Construction & Landscaping,Print Shop,Hockey Arena,Furniture / Home Store,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
11,Chomedey Northwest,0.0,Grocery Store,Drugstore,Ice Cream Shop,Sporting Goods Shop,Pizza Place,Fast Food Restaurant,Coffee Shop,Automotive Shop,English Restaurant,Electronics Store
12,Chomedey South,0.0,Grocery Store,Dessert Shop,Gym / Fitness Center,Bank,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Yoga Studio
13,Outremont,0.0,Park,Bakery,Restaurant,Metro Station,Pizza Place,Gourmet Shop,American Restaurant,Ice Cream Shop,Spanish Restaurant,Italian Restaurant


In [197]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 1, montreal_merged.columns[[1] + list(range(6, montreal_merged.shape[1]))]]

,Place,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Duvernay-Est,1.0,Convenience Store,Sushi Restaurant,Pharmacy,Drugstore,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Pont-Viau,1.0,Breakfast Spot,Indian Restaurant,Convenience Store,Supermarket,Park,Curling Ice,Cycle Studio,Creperie,Flower Shop,Fish Market
4,Auteuil West,1.0,ATM,Pharmacy,Grocery Store,Convenience Store,Curling Ice,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant
16,RiviÃ¨re-Des-Prairies Southwest,1.0,Pharmacy,Convenience Store,Fast Food Restaurant,Italian Restaurant,Grocery Store,Yoga Studio,English Restaurant,Eastern European Restaurant,Electronics Store,Empada House
20,Anjou East,1.0,Convenience Store,Pizza Place,BBQ Joint,American Restaurant,Hockey Arena,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant
28,Rosemont Central,1.0,ATM,Bus Station,Park,Pet Store,Pharmacy,Deli / Bodega,Portuguese Restaurant,Chinese Restaurant,Convenience Store,Arts & Crafts Store
30,Saint-Michel West,1.0,Convenience Store,Pharmacy,Department Store,Park,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Dumpling Restaurant
50,L'Ãle-Des-Soeurs,1.0,Convenience Store,Playground,Mediterranean Restaurant,Park,Lake,Cycle Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant
56,Mount Royal Central,1.0,Park,Portuguese Restaurant,Sushi Restaurant,Flower Shop,Cupcake Shop,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
59,CÃ´te-des-Neiges East,1.0,Convenience Store,Business Service,Bus Station,Hobby Shop,Herbs & Spices Store,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Falafel Restaurant


In [198]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 2, montreal_merged.columns[[1] + list(range(6, montreal_merged.shape[1]))]]

,Place,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Laval-des-Rapides,2.0,Park,Scenic Lookout,Business Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
38,Ahuntsic East,2.0,Park,Athletics & Sports,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
52,Downtown Montreal South & West,2.0,Historic Site,Park,Mountain,Scenic Lookout,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
61,Westmount West,2.0,Park,Athletics & Sports,Tennis Court,Scenic Lookout,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Drugstore,Empanada Restaurant
67,Verdun South,2.0,Park,Construction & Landscaping,Playground,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
69,Cartierville Southwest,2.0,Park,Empada House,Tailor Shop,Yoga Studio,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
74,Saint-Laurent Central,2.0,Park,Yoga Studio,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
84,Lachine West,2.0,Park,Bank,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [199]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 3, montreal_merged.columns[[1] + list(range(6, montreal_merged.shape[1]))]]

,Place,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Montreal East,3.0,Recording Studio,Yoga Studio,Drugstore,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [200]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 4, montreal_merged.columns[[1] + list(range(6, montreal_merged.shape[1]))]]

,Place,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,RiviÃ¨re-des-Prairies Northeast,4.0,Construction & Landscaping,Bus Stop,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
89,L'Ãle-Dorval,4.0,Bus Stop,Locksmith,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


## Section 3: Toronto Zip Codes Clustering

In [118]:
torgeodf = torgeodf.drop(columns = ['Unnamed: 0'])

In [120]:
torgeodf.columns = ['Code', 'Admin', 'Place', 'Latitude', 'Longitude']

In [121]:
torgeodf.head()

,Code,Admin,Place,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [122]:
radius = 500
LIMIT = 100

venuestor = []

for lat, long, code, admin, place in zip(torgeodf['Latitude'], torgeodf['Longitude'], torgeodf['Code'], torgeodf['Admin'], torgeodf['Place']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venuestor.append((
            code, 
            admin,
            place,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [124]:
venuestordf = pd.DataFrame(venuestor)

venuestordf.columns = ['PostalCode', 'Admin', 'Place', 'PlaceLatitude', 'PlaceLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venuestordf.shape)
venuestordf.head()

(2132, 9)


,PostalCode,Admin,Place,PlaceLatitude,PlaceLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [126]:
venuestordf.groupby(["Place"]).count()

,PostalCode,Admin,PlaceLatitude,PlaceLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Place,,,,,,,,
Agincourt,5,5,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56,56,56
"Birch Cliff, Cliffside West",4,4,4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16,16,16


In [128]:
len(venuestordf['VenueCategory'].unique())

280

In [129]:
# one hot encoding
toronto_onehot = pd.get_dummies(venuestordf[['VenueCategory']], prefix="", prefix_sep="")

# add 'place' column back to dataframe
toronto_onehot['Place'] = venuestordf['Place']

toronto_onehot = toronto_onehot.set_index('Place').reset_index()

In [130]:
toronto_onehot.head()

,Place,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern

In [132]:
toronto_onehot.shape

(2132, 281)

In [133]:
toronto_grouped = toronto_onehot.groupby(["Place"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(95, 281)


,Place,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincour

In [134]:
num_top_venues = 5

for place in toronto_grouped['Place']:
    print("----"+place+"----")
    temp = toronto_grouped[toronto_grouped['Place'] == place].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Clothing Store   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1            Pharmacy  0.11
2                 Gym  0.11
3  Athletics & Sports  0.11
4        Dance Studio  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2                Gas Station  0.05
3                      Diner  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                        venue  freq
0         Japanese Restaurant  0.25
1                        Bank  0.25
2          Chinese Restaurant  0.25
3                        Café  0.25
4  Modern European Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                v

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [136]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Place']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


places_venuestor_sorted = pd.DataFrame(columns=columns)
places_venuestor_sorted['Place'] = toronto_grouped['Place']

for ind in np.arange(toronto_grouped.shape[0]):
    places_venuestor_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

places_venuestor_sorted.head()

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Sandwich Place,Pharmacy,Coffee Shop,Athletics & Sports,Pub,Doner Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Ice Cream Shop,Supermarket,Sushi Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Deli / Bodega,Fried Chicken Joint,Diner
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Greek Restaurant,Thai Restaurant,Pharmacy,Pizza Place,Pub,Café


In [137]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Place', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [138]:
# add clustering labels
places_venuestor_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [139]:
toronto_merged = torgeodf


toronto_merged = toronto_merged.join(places_venuestor_sorted.set_index('Place'), on='Place')

In [141]:
toronto_merged = toronto_merged.dropna()

toronto_merged = toronto_merged.reset_index(drop=True)

In [145]:
# create map
map_clusterstor = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Place'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusterstor)
       
map_clusterstor

### Examin Toronto Cluster

In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Admin,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,East York,0.0,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Pharmacy,Café,Bank,Gym / Fitness Center
70,North York,0.0,Pizza Place,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Pub,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant
79,York,0.0,Pizza Place,Caribbean Restaurant,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
87,Etobicoke,0.0,Pizza Place,Dance Studio,Gym,Sandwich Place,Pharmacy,Coffee Shop,Athletics & Sports,Pub,Doner Restaurant,Diner
92,North York,0.0,Pizza Place,Furniture / Home Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant,Drugstore
95,Etobicoke,0.0,Pizza Place,Discount Store,Intersection,Sandwich Place,Chinese Restaurant,Coffee Shop,Middle Eastern Restaurant,Donut Shop,Distribution Center,Dog Run
96,Etobicoke,0.0,Sandwich Place,Pizza Place,Mobile Phone Shop,Bus Line,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore
97,Etobicoke,0.0,Grocery Store,Pizza Place,Pharmacy,Fried Chicken Joint,Beer Store,Sandwich Place,Fast Food Restaurant,Construction & Landscaping,Comic Shop,Falafel Restaurant


In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Admin,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,Scarborough,1.0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,Scarborough,1.0,Breakfast Spot,Intersection,Bank,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Yoga Studio,Drugstore,Doner Restaurant
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,Scarborough,1.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Hakka Restaurant,Electronics Store,Eastern European Restaurant
5,Scarborough,1.0,Playground,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
6,Scarborough,1.0,Convenience Store,Department Store,Bus Station,Coffee Shop,Discount Store,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,1.0,Bus Line,Bakery,Bus Station,Ice Cream Shop,Intersection,Metro Station,Soccer Field,Park,Eastern European Restaurant,Dumpling Restaurant
8,Scarborough,1.0,Motel,American Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
9,Scarborough,1.0,College Stadium,General Entertainment,Skating Rink,Café,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Admin,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,Etobicoke,2.0,Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Discount Store
93,North York,2.0,Baseball Field,Food Service,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Admin,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,3.0,Cafeteria,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Discount Store


In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Admin,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4.0,Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
21,North York,4.0,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
23,North York,4.0,Food & Drink Shop,Fireworks Store,Park,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
38,East York,4.0,Park,Intersection,Convenience Store,Metro Station,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
42,Central Toronto,4.0,Park,Bus Line,Swim School,Filipino Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
46,Central Toronto,4.0,Gym,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
48,Downtown Toronto,4.0,Park,Playground,Trail,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
72,York,4.0,Park,Women's Store,Pool,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
88,Etobicoke,4.0,Park,River,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
94,York,4.0,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


## Section 4: Montreal and Toronto Clustering as One Dataset

In [151]:
venuesmt = pd.concat([venuesdf, venuestordf])

In [165]:
venuesmt.shape

(3582, 9)

In [156]:
# one hot encoding
mt_onehot = pd.get_dummies(venuesmt[['VenueCategory']], prefix="", prefix_sep="")

# add Plac column back to dataframe
mt_onehot['Place'] = venuesmt['Place']

In [157]:
mt_onehot = mt_onehot.set_index('Place').reset_index()

In [168]:
mt_onehot.head()

,Place,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Bookstore,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Sausage Shop,Scenic Lookout,Scien

In [159]:
mt_onehot.shape

(3582, 337)

In [161]:
mt_grouped = mt_onehot.groupby(["Place"]).mean().reset_index()

print(mt_grouped.shape)
mt_grouped.head()

(186, 337)


,Place,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Bookstore,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Sausage Shop,Scenic Lookout,Scien

In [171]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Place']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


places_venuestor_sorted_mt = pd.DataFrame(columns=columns)
places_venuestor_sorted_mt['Place'] = mt_grouped['Place']

for ind in np.arange(mt_grouped.shape[0]):
    places_venuestor_sorted_mt.iloc[ind, 1:] = return_most_common_venues(mt_grouped.iloc[ind, :], num_top_venues)

places_venuestor_sorted_mt.head()

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
1,Ahuntsic Central,Café,Pharmacy,Ice Cream Shop,Pizza Place,Sushi Restaurant,Bar,Bakery,Cheese Shop,Dessert Shop,Restaurant
2,Ahuntsic East,Park,Athletics & Sports,Yoga Studio,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Ahuntsic Southeast,Furniture / Home Store,Italian Restaurant,Breakfast Spot,Department Store,Bakery,Sandwich Place,Café,Liquor Store,Discount Store,Electronics Store
4,Ahuntsic Southwest,Pharmacy,Ice Cream Shop,Beer Store,New American Restaurant,Breakfast Spot,Café,Liquor Store,Restaurant,Lounge,Grocery Store


In [172]:
# set number of clusters
kclusters = 5

mt_grouped_clustering = mt_grouped.drop('Place', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 1, 2, 2, 1, 2, 2], dtype=int32)

In [173]:
# add clustering labels
places_venuestor_sorted_mt.insert(0, 'Cluster Labels', kmeans.labels_)

In [176]:
tor = torgeodf.drop(columns = ['Admin'])

In [178]:
mont = dfmontreal2.drop(columns = ['Province', 'Admin'])

In [182]:
mt = pd.concat([tor, mont])

In [184]:
mt_merged = mt


mt_merged = mt_merged.join(places_venuestor_sorted_mt.set_index('Place'), on='Place')

In [186]:
mt_merged = mt_merged.dropna()

mt_merged = mt_merged.reset_index(drop=True)

In [208]:
# create map
map_clustersmt = folium.Map(location=[44.233334, -76.500000], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mt_merged['Latitude'], mt_merged['Longitude'], mt_merged['Place'], mt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clustersmt)
       
map_clustersmt

In [191]:
mt_merged.loc[mt_merged['Cluster Labels'] == 0, mt_merged.columns[[1] + list(range(5, mt_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Milliken, Agincourt North, Steeles East, L'Amo...",Playground,Park,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
21,York Mills West,Convenience Store,Park,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
23,Parkwoods,Fireworks Store,Food & Drink Shop,Park,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
38,"East Toronto, Broadview North (Old East York)",Metro Station,Convenience Store,Park,Intersection,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
42,Lawrence Park,Bus Line,Swim School,Park,Yoga Studio,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
46,"Moore Park, Summerhill East",Park,Gym,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
48,Rosedale,Park,Playground,Trail,Yoga Studio,Empada House,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
72,Caledonia-Fairbanks,Park,Women's Store,Pool,Historic Site,Fireworks Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
88,"The Kingsway, Montgomery Road, Old Mill North",Park,River,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
94,Weston,Park,Convenience Store,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


In [192]:
mt_merged.loc[mt_merged['Cluster Labels'] == 1, mt_merged.columns[[1] + list(range(5, mt_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",Mexican Restaurant,Breakfast Spot,Medical Center,Bank,Electronics Store,Rental Car Location,Intersection,English Restaurant,Drugstore,Dumpling Restaurant
3,Woburn,Coffee Shop,Korean Restaurant,English Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House
4,Cedarbrae,Gas Station,Caribbean Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Bank,Hakka Restaurant,Fried Chicken Joint,Yoga Studio,Electronics Store
6,"Kennedy Park, Ionview, East Birchmount Park",Coffee Shop,Convenience Store,Discount Store,Bus Station,Department Store,English Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
7,"Golden Mile, Clairlea, Oakridge",Bus Line,Bakery,Ice Cream Shop,Intersection,Bus Station,Park,Metro Station,Soccer Field,Empanada Restaurant,Drugstore
8,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant
9,"Birch Cliff, Cliffside West",Skating Rink,Café,College Stadium,General Entertainment,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
10,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Pet Store,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
11,"Wexford, Maryvale",Middle Eastern Restaurant,Sandwich Place,Bakery,Auto Garage,Breakfast Spot,Fireworks Store,Filipino Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
12,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


In [193]:
mt_merged.loc[mt_merged['Cluster Labels'] == 2, mt_merged.columns[[1] + list(range(5, mt_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,English Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House
13,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Rental Car Location,Bank,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Fried Chicken Joint,Gas Station,Fast Food Restaurant
22,"Willowdale, Willowdale West",Coffee Shop,Bank,Pizza Place,Pharmacy,Butcher,Grocery Store,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
33,"Parkview Hill, Woodbine Gardens",Pizza Place,Gastropub,Fast Food Restaurant,Pharmacy,Pet Store,Bank,Café,Athletics & Sports,Gym / Fitness Center,Intersection
70,Glencairn,Pizza Place,Pub,Asian Restaurant,Japanese Restaurant,Sushi Restaurant,Yoga Studio,Empada House,Doner Restaurant,Donut Shop,Drugstore
79,"Runnymede, The Junction North",Convenience Store,Caribbean Restaurant,Pizza Place,Yoga Studio,English Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
87,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Pharmacy,Athletics & Sports,Gym,Dance Studio,Electronics Store,Dog Run
92,Humber Summit,Pizza Place,Furniture / Home Store,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
95,Westmount,Pizza Place,Discount Store,Coffee Shop,Sandwich Place,Chinese Restaurant,Intersection,Middle Eastern Restaurant,Creperie,Cuban Restaurant,Donut Shop
96,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Mobile Phone Shop,Pizza Place,Bus Line,Yoga Studio,Empada House,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


In [194]:
mt_merged.loc[mt_merged['Cluster Labels'] == 3, mt_merged.columns[[1] + list(range(5, mt_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"York Mills, Silver Hills",Cafeteria,Yoga Studio,English Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant


In [195]:
mt_merged.loc[mt_merged['Cluster Labels'] == 4, mt_merged.columns[[1] + list(range(5, mt_merged.shape[1]))]]

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",Construction & Landscaping,Bar,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant
5,Scarborough Village,Construction & Landscaping,Playground,Yoga Studio,English Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
114,RiviÃ¨re-des-Prairies Northeast,Construction & Landscaping,Bus Stop,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empada House,Empanada Restaurant
